In [134]:
# coding: utf-8
import os, sys, itertools, random, sqlite3
import pandas as pd

teams = ['ATL','BOS','BKN','CHA','CHI','CLE','DAL','DEN','DET','GSW','HOU','IND','LAC','LAL','MEM','MIA','MIL','MIN','NOP','NYK','OKC','ORL','PHI','PHX','POR','SAC','SAS','TOR','UTA','WAS']
#北京时间。换成美国东部时间须减一天
#NBAseason = ['2010-10-27', '2011-04-14', '2011-12-26', '2012-04-27', '2012-10-31', '2013-04-18', '2013-10-30', '2014-04-17', '2014-10-29', '2015-04-16', '2015-10-28', '2016-04-14', '2016-10-26', '2017-04-13', '2017-10-18', '2018-04-XX']

sqlPath = os.getcwd().replace('jupyter_notebook', 'web_spiders\\NBA_from_2010.db')
connection = sqlite3.connect(sqlPath)
cursor = connection.cursor()
table_name = 'NBA_scores_from_1998'

#去重
# cursor.execute('DELETE FROM %s WHERE rowid NOT IN (SELECT MAX(rowid) rowid FROM %s GROUP BY game_ID)'%(table_name,table_name))
# connection.commit()

def seasonGame(startDate, endDate):
    cursor.execute('SELECT game_date_EST,road_team,home_team,road_points,home_points FROM {} WHERE strftime("%Y-%m-%d",game_date_EST) >= "{}" AND strftime("%Y-%m-%d",game_date_EST) <= "{}"'.format(table_name,startDate,endDate))
    sql0 = cursor.fetchall()
    col_name_list = ['game_date_EST', 'road_team', 'home_team', 'road_points', 'home_points'] #[tuple[0] for tuple in cursor.description]
    df = pd.DataFrame(sql0, columns=col_name_list)
    df['total_points'] = df['road_points'] + df['home_points']
    return df

df = seasonGame('2016-10-25','2017-04-14')

In [135]:
def teamOdevity(targetTeam,startLoc=0,testLength=df.shape[0]):
    df1 = df[(df['road_team']==targetTeam)|(df['home_team']==targetTeam)]
    # df1 = df[df['road_team']==targetTeam]
    # df1 = df[df['home_team']==targetTeam]
    odevity = list(df1['total_points'].apply(lambda x:x%2))
    return odevity[startLoc:startLoc + testLength]

In [136]:
def cartesian(patternContent,patternLength):
    cartesian = []
    for i in itertools.product(patternContent, repeat=patternLength):
        cartesian.append(i)
    return cartesian

In [137]:
def odevityProfit(odevityList,pattern,timesCast):#timesCast:2倍 or 3倍投
    totalcostRatio = (timesCast**len(pattern)-1)/(timesCast-1)#单位总成本
    totalAmount = 10000 #起始总金额
    initBet = totalAmount//(2*totalcostRatio) #取总金额与单位总成本的2倍的商作为初始投注值；2倍：1 3 7 15; 3倍：1 4 13 40 121 364 ……
    odevityIndex = 1 #列表的初始比对位置
    while odevityIndex <= len(odevityList) - len(pattern):
        a = odevityList[odevityIndex - 1]#取前一位的值为输入
        b = 1 - a
        betPattern = [a if i =='a' else b for i in pattern]
        patternIndex = 0 #pattern中的相对位置
        while patternIndex < len(pattern):
            income = initBet*1.9*timesCast**(patternIndex)
            cost = initBet*(timesCast**(patternIndex+1)-1)/(timesCast-1)
            profit = income - cost
            if betPattern[patternIndex] == odevityList[patternIndex + odevityIndex]:
                totalAmount += profit
                #print(patternIndex)
                #print('       ' + str(totalAmount))
                break
            elif patternIndex == len(pattern) - 1 and betPattern[patternIndex] != odevityList[patternIndex + odevityIndex]:
                totalAmount -= cost
                #print(patternIndex)
                #print('       ' + str(totalAmount))
                break
            else:
                patternIndex += 1

        odevityIndex += patternIndex + 1
        #print('   ' + str(odevityIndex))
    return totalAmount

In [145]:
cartesian0 = [''.join(i) for i in cartesian('ab',2)]
ndarray = []
for i in teams:
    targetOdevity = teamOdevity(i)
    #print(len(targetOdevity))#(''.join([str(i) for i in targetOdevity]))
    array1 = []
    for j in cartesian0:
        k = odevityProfit(targetOdevity,j,2)
        array1.append(k)
    ndarray.append(array1)
df0 = pd.DataFrame(ndarray,index=teams,columns=cartesian0).T
pd.set_option('display.max_columns',30)
df0[df0>10000].fillna('-')
#sum(df0[df0>10000].count())


,ATL,BOS,BKN,CHA,CHI,CLE,DAL,DEN,DET,GSW,HOU,IND,LAC,LAL,MEM,MIA,MIL,MIN,NOP,NYK,OKC,ORL,PHI,PHX,POR,SAC,SAS,TOR,UTA,WAS
aa,24994,-,-,-,-,-,34157,-,-,-,34656.8,-,-,-,-,-,-,29492.2,-,-,-,-,12499,-,-,-,29992,11832.6,-,-
ab,-,-,31324.8,24827.4,-,-,-,-,32824.2,-,-,-,-,-,16997.2,24994,-,-,-,43986.4,20162.6,-,37822.2,23328,-,16664,17330.4,-,27826.2,-
ba,26493.4,-,-,13665.2,-,-,-,-,-,-,15331.2,10666.4,-,-,-,-,-,13998.4,57314.4,-,-,-,-,-,-,-,-,10999.6,-,21995.2
bb,-,37489,-,-,28159.4,-,28326,24994,-,26493.4,-,-,15164.6,34656.8,-,-,20162.6,13998.4,38322,-,-,18496.6,-,-,-,-,-,36322.8,-,15664.4
